In [3]:
from draughts import Draughts
import random
import sys

### Define heuristic players for MCTS

* **RandomPlayer** – selects moves entirely at random using the `random.choice()` function.
* **RandomPlayerCaptureIfPossible** – behaves like `RandomPlayer`, but prioritizes capturing opponent pieces when possible.


In [4]:
class RandomPlayer:
    def __init__(self, name="Random Player"):
        self.name = name

    def move(self, board, player_id, valid_moves):
        a = random.choice(valid_moves)
        return a

class RandomPlayerCaptureIfPossible:
    def __init__(self, name="Random Player Capturer"):
        self.name = name   

    def move(self, board, player_id, valid_moves):
        capture_moves = [move for move in valid_moves if move['captures']]
        if len(capture_moves) > 0:
            return random.choice(capture_moves)
        return random.choice(valid_moves)


player1 = RandomPlayer()
player2 = RandomPlayerCaptureIfPossible()
# Draughts(player1, player1).play()
results = {player1.name: 0, player2.name: 0}
for i in range(10):
    game = Draughts(player1, player2)
    result = game.play(verbose=False)
    if result == -1:
        results[player1.name] += 1
    else:
        results[player2.name] += 1
    game = Draughts(player2, player1)
    result = game.play(verbose=False)
    if result == -1:
        results[player2.name] += 1
    else:
        results[player1.name] += 1
print(results)
    


{'Random Player': 17, 'Random Player Capturer': 3}


It seems that player who plays all at random is better than player who seeks for capture.

### Implementation of **Monte Carlo Tree Search**

The algorithm follows four main steps:

* **Selection** – traverse through *n* random possible moves (all starting from the default state).
* **Expansion** – create *n* instances of the game (if no winning move is found).
* **Playout** – simulate the games using `game.play()` with the given `heuristics_player`, calculating the likelihood of winning.
* **Backpropagation** – choose the move with the highest likelihood of winning (or select the immediate winning move, if available).


In [5]:
class MonteCarloTreeSearchPlayer:
    def __init__(self, name="MCTS Player", heuristics_player=RandomPlayer(), n=None, simulations=100):
        self.name = name
        self.heuristics_player = heuristics_player
        self.n = n
        self.simulations = simulations

    def calculate_likelihood(self, move, board, player_id):
        wins = 0
        game = Draughts(self.heuristics_player, self.heuristics_player)
        for _ in range(self.simulations):
            game.set_board(board, player_id)
            game.make_move(move)
            
            # if game is over, curent player wins so likelihood is 100%
            if game.check_for_end():
                return 1
            
            result = game.play(verbose=False)
            if result == player_id:
                wins += 1
        return wins / self.simulations

    def move(self, board, player_id, valid_moves):
        
        if self.n is None or self.n <= len(valid_moves):
            compute_moves = valid_moves
        else:
            compute_moves = random.sample(valid_moves, self.n)

        best_move = None
        best_likelihood = 0
        for move in compute_moves:
            likelihood = self.calculate_likelihood(move, board, player_id)

            # if there is a winning move, return it
            if likelihood == 1:
                return move
            # if there is no winning move, seach for the best move
            if likelihood > best_likelihood:
                best_likelihood = likelihood
                best_move = move
        return best_move



In [6]:
Draughts(MonteCarloTreeSearchPlayer(), RandomPlayer()).play()

    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
5 │ ● │   │ ● │   │ ● │   │ ● │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
6 │   │ ● │   │ ● │   │ ● │   │ ● │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
7 │ ● │   │ ● │   │ ● │   │ ● │   │


    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │   │   │   │   │ ● │ 

-1

In [ ]:
player1 = MonteCarloTreeSearchPlayer()
player2 = RandomPlayer()
results = {player1.name: 0, player2.name: 0}
print(f"Starting {20} games, comparing {player1.name} vs {player2.name}...")
for i in range(10):
    print(f"Game {i+1}/20: {player1.name} (player 1) vs {player2.name} (player 2)...", end=" ")
    game = Draughts(player1, player2)
    result = game.play(verbose=False)
    if result == -1:
        results[player1.name] += 1
        print(f"{player1.name} wins!")
    else:
        results[player2.name] += 1
        print(f"{player2.name} wins!")
for i in range(10):
    print(f"Game {i+11}/20: {player2.name} (player 1) vs {player1.name} (player 2)...", end=" ")
    game = Draughts(player2, player1)
    result = game.play(verbose=False)
    if result == -1:
        results[player2.name] += 1
        print(f"{player2.name} wins!")
    else:
        results[player1.name] += 1
        print(f"{player1.name} wins!")
print(results)
    

Starting 20 games, comparing MCTS Player vs Random Player...
Game 1/20: MCTS Player (player 1) vs Random Player (player 2)... 